<a href="https://colab.research.google.com/github/ftk1000/5G/blob/master/parse_3gpp_v2b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parse 3GPP document
2020.08.19

parse_3gpp_v2b.ipynb

In [24]:
# -*- coding: utf-8 -*-
# parse_3gpp_v2.py
# 2020.08.19
#---------------------------------------------
#  README:
#---------------------------------------------
#prepare data set : parse 3GPP 36523-1-f00_s07_01.txt and 
#extract sentences corresponding to "test case name", "with", "when", "then"
#
# SOURCE DOCUMENT
# LTE; Evolved Universal Terrestrial Radio Access (E-UTRA) and Evolved Packet Core (EPC);
# User Equipment (UE) conformance specification; Part 1: Protocol conformance specification
# 3GPP TS 36.523-1

# downloaded *.DOC files are here:  36523-1-f00.zip
# we will use only of them converted to TXT format
#
#INPUT DATA:
# 36523-1-f00_s07_01.txt
#
#36523-1-f00_s07_01.txt files was generated from 
#36523-1-f00_s07_01.DOC file located in
#
#OUTPUT:
# 36523-1-f00_s07_01_tests.csv   

import numpy as np
import pandas as pd
import string
import random

# Authenticate, create the PyDrive client, and READ DATA

In [15]:
### Here are the instructions how to read a file from Gdrive
# https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/

# shareble_link = 'https://drive.google.com/file/d/1jNGAf5O6U4Cr4GbWK_DpCxIuxv7-BQ2c/view?usp=sharing';  file_id = '1jNGAf5O6U4Cr4GbWK_DpCxIuxv7-BQ2c'
# shareble_link = 'https://drive.google.com/file/d/0B1Iyay9XxksvMG9wLUpFQm1Hbnc/view?usp=sharing';  file_id = '0B1Iyay9XxksvMG9wLUpFQm1Hbnc'
shareble_link = 'https://drive.google.com/file/d/18agBYxgWJ4dVZGOYK1mil1RDhjvQL8e1/view?usp=sharing'; file_id = '18agBYxgWJ4dVZGOYK1mil1RDhjvQL8e1'

!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [16]:
#see Step 6 https://buomsoo-kim.github.io/colab/2018/04/16/Importing-files-from-Google-Drive-in-Google-Colab.md/
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [17]:
downloaded = drive.CreateFile({'id':file_id})   # replace the id with id of file you want to access
downloaded.GetContentFile('36523-1-f00_s07_01.zip') 

In [20]:
# !ls -l
!unzip 36523-1-f00_s07_01.zip

Archive:  36523-1-f00_s07_01.zip
replace 36523-1-f00_s07_01.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: 36523-1-f00_s07_01.txt  


In [21]:
# !more RespondentTypeREADME.txt
wd = './'
file_name = wd+'36523-1-f00_s07_01.txt'

In [27]:
# to deal with non-ascii codes read this
# https://stackoverflow.com/questions/22216076/unicodedecodeerror-utf8-codec-cant-decode-byte-0xa5-in-position-0-invalid-s
# import codecs      # f = codecs.open(file_name, "r", encoding= 'unicode_escape')
# file_name = '36523-1-f00_s07_01.txt'
with open(file=file_name, mode='r', encoding='unicode_escape') as f:
    lines = f.readlines()
f.close()

printable = set(string.printable)
lines2 = [  ''.join(filter(lambda x: x in printable, s)) for s in lines  ]
assert len(lines2)==len(lines)
print(len(lines))

102056


# TAKE A LOOK AT DATA

In [42]:
# Show a few lines describing a test case
k=218
lines[k-4:k+10]

['\n',
 '7.1.1.1.3.3\tSpecific message contents\n',
 'None.\n',
 '7.1.1.1a\tCCCH mapped to UL SCH/ DL-SCH / UE Cat 0\n',
 '7.1.1.1a.1\tTest Purpose (TP)\n',
 '(1)\n',
 'with { UE in E-UTRA RRC_IDLE state }\n',
 'ensure that {\n',
 '  when { UE receives a Paging message including a matched identity }\n',
 '    then { UE responds with a RRCConnectionRequest message in a MAC PDU on UL SCH on CCCH indicating LCID \x9201011\x92  }\n',
 '            }\n',
 '\n',
 '7.1.1.1a.2\tConformance requirements\n',
 'References: The conformance requirements covered in the current TC are specified in: TS 36.321, clause 5.3.3, 5.11, 6.1.2 and 6.2.1.\n']

In [43]:
# Some 'with' are followed by '{', some by '('
print(lines[220].strip())
print(lines[16000].strip())

with { UE in E-UTRA RRC_IDLE state }
with ( UE in E-UTRA RRC_CONNECTED state )


# PARSING/SEARCH/REGEX FUNCTIONS

In [44]:
# find line numbers that contain PATTERN
def find_lineNums(lines, pattern):
    test_idx = []
    for i in range(len(lines)):
      line = lines[i].strip();  # print(line)
      if line.find(pattern) >=0 :
        test_idx.append(i)
    return test_idx

PATTERN = "Test Purpose (TP)"
test_idx = find_lineNums(lines=lines, pattern= PATTERN)    
print(test_idx)
print("patterns found =",len(test_idx))

[36, 218, 352, 508, 873, 1087, 1243, 1596, 1949, 2241, 2621, 2782, 2973, 3196, 3435, 3725, 4275, 5016, 5605, 5754, 6360, 6621, 6707, 6933, 7193, 7507, 7807, 8127, 8250, 8544, 9076, 9544, 10221, 10660, 11099, 11743, 12380, 12620, 13509, 13656, 14067, 14273, 15027, 15295, 15383, 15532, 15998, 16568, 16809, 17228, 17548, 17768, 18205, 18605, 18918, 19220, 19883, 20194, 20455, 20758, 21173, 21646, 21989, 22399, 22661, 23370, 23385, 23748, 24075, 24312, 24649, 24963, 25424, 25460, 25765, 26315, 26532, 26903, 27753, 27920, 28137, 28320, 28428, 28535, 28926, 29245, 29564, 29878, 30213, 30548, 34821, 38947, 43126, 47278, 51785, 56037, 61926, 67310, 72686, 78107, 83529, 84824, 86116, 92189, 93231, 94224, 98418, 99255, 99901, 100314, 100705, 101044, 101333, 101881]
patterns found = 114


In [45]:
import re
R = re.compile(r"^\(\d+\)")

#for line in lines:
#    M = R.search(line)
#    if M:
#        print(M.string)
# keep looking for '(\d)' until find 'Conformance requirements'
# or the next test (ix2)
def find_idx_subtests(ix1,ix2):
  ix_subtest = []
  for i in range(ix1,ix2):
    M = R.search(lines[i])
    if M:
      #lines[i][0]=='(':
      ix_subtest.append(i)
      # print('==> lineNum =',i, "  line=", lines[i].strip())
    if lines[i].find("Conformance requirements") >=0 :
        ix_subtest.append(i)   #conf_req_line = i
        break;
  return ix_subtest
#%%
k=46
ix1=test_idx[k]; ix2=test_idx[k+1]
subtests = find_idx_subtests(ix1, ix2)  
for p in range(len(subtests)):
    print(subtests[p], '    ',lines[subtests[p]].strip() )

15999      (1)
16006      (2)
16013      (3)
16020      (4)
16026      (5)
16033      (6)
16039      (7)
16045      (8)
16052      7.1.4.6.2	Conformance requirements


# BUILD A DATA FRAME

In [111]:
d = pd.DataFrame(columns=['doc','test_sec', 'test_name', 'test_num',  'with', 'when', 'then'])
doc_name = file_name.split('.')[0]
#d.loc[0] = ['7.1.1.1',  'CCCH mapped to','1', 'with aa', 'when bb', 'then cc'] 

# BUILD THE DATA BASE
df_index = 0
for k in range(len(test_idx)-1):
    ix=test_idx[k]
    ix2=test_idx[k+1]
    #def update_df(ix):
    [sec_num, test_name] = lines[ix-1].strip().split('\t')
    ix_subtests = find_idx_subtests(ix,ix2)
    for p in range(len(ix_subtests)-1):
        index = ix_subtests[p]
        
        with_kw = when_kw = then_kw = ''
        while index < ix_subtests[p+1]:
            line = lines[index].strip()
            if line.find('with') >=0 :
                with_kw = line[len('with'):].strip()[1:-1].strip()
            if line.find('when') >=0 :
                when_kw = line[len('when'):].strip()[1:-1].strip()
            if line.find('then') >=0 :
                then_kw = line[len('then'):].strip()[1:-1].strip()
            index += 1
            
        d.loc[df_index] = [doc_name, sec_num,  test_name, p+1, with_kw, when_kw, then_kw] 
        df_index += 1

print(d.shape)
d.head()

(279, 7)


,doc,test_sec,test_name,test_num,with,when,then
0,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,1,UE in E-UTRA RRC_IDLE state and after transmit...,UE receives a MAC PDU on DL SCH and addressed ...,UE discards the MAC PDU
1,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,2,UE receives a MAC PDU on DL SCH and addressed ...,UE receives a MAC PDU on DL SCH and addressed ...,UE forwards to upper layers the disassembled a...
2,36523-1-f00_s07_01,7.1.1.1a,CCCH mapped to UL SCH/ DL-SCH / UE Cat 0,1,UE responds with a RRCConnectionRequest messag...,UE receives a Paging message including a match...,UE responds with a RRCConnectionRequest messag...
3,36523-1-f00_s07_01,7.1.1.2,DTCH or DCCH mapped to UL SCH/ DL-SCH / Reserv...,1,UE in E-UTRA RRC_Connected state with DRB [Log...,UE receives a MAC PDU on DL SCH and addressed ...,UE shall not forward the disassembled and demu...
4,36523-1-f00_s07_01,7.1.1.2,DTCH or DCCH mapped to UL SCH/ DL-SCH / Reserv...,2,UE receives a MAC PDU on DL SCH and addressed ...,UE receives a MAC PDU on DL SCH and addressed ...,UE shall forward the disassembled and demultip...


In [112]:
wd='./'
d.to_csv(wd+doc_name+'_tests.csv')

In [113]:
# #%% GENERATE TRAIN AND TEST SETS
# import numpy as np
# txt = np.array(list(d['with'])    + list(d['when'])    + list(d['then']))
# lbl = np.array([0]*len(d['with']) + [1]*len(d['when']) + [2]*len(d['then']) )

# assert len(lbl)==len(txt)

# # Shuffle and split
# idx = np.random.permutation(len(txt))
# x,y = txt[idx], lbl[idx]
# df=pd.DataFrame({'sentence':x,'label':y})

# SPLIT DATA FRAME INTO TRAIN AND TEST

In [118]:
np.random.seed(seed=60)
# idx = np.random.permutation(d.shape[0])

test_rows_num = 5
# test_rows = random.sample( range(0,d.shape[0]), test_rows_num ) 
# test_rows=[0,1,2]
test_rows= [156, 253, 209, 43, 271]
print('test_rows=', test_rows)
d_test = d.iloc[test_rows]
d_train = d.drop(test_rows)

def get_X_Y(d):
    txt = np.array(list(d['with'])    + list(d['when'])    + list(d['then']))
    lbl = np.array([0]*len(d['with']) + [1]*len(d['when']) + [2]*len(d['then']) )
    return txt, lbl

idx = np.random.permutation(len(d_train))
X_train, y_train = get_X_Y(d_train.iloc[idx])
X_test,  y_test  = get_X_Y(d_test)

assert(len(X_train)==len(y_train))
assert(len(X_test)==len(y_test))
print(len(y_train))
print(len(y_test))

test_rows= [156, 253, 209, 43, 271]
822
15


In [119]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)   # Plain Train set
X_test_counts  = count_vect.transform(X_test)

In [120]:
#%% XGBoost: XGB 
import xgboost as xgb
import warnings
warnings.filterwarnings(module='xgboost*', action='ignore', category=DeprecationWarning)

xgbmodel = xgb.XGBClassifier( max_depth=1, n_estimators=3000, learning_rate=0.005 ).fit(X_train_counts, y_train)
predicted = xgbmodel.predict(X_test_counts)
res=np.mean(predicted == y_test)
print(res)
#oac_df.loc['XGB']=res

1.0


In [121]:
# print(X_test)
print(y_test)
print(predicted)

def compare_tuples(test_rows_num, y_test, predicted):
  test_tuples = []
  pred_tuples = []
  for k in range(test_rows_num):
    test_tuples.append([y_test[k], y_test[k+test_rows_num], y_test[k+2*test_rows_num]])
    pred_tuples.append([predicted[k], predicted[k+test_rows_num], predicted[k+2*test_rows_num]])

  d_tuples = pd.DataFrame( {'test_tuples':test_tuples, 'pred_tuples':pred_tuples} )
  d_tuples['match']=d_tuples['test_tuples']==d_tuples['pred_tuples']
  return d_tuples

compare_tuples(test_rows_num, y_test, predicted)

[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]
[0 0 0 0 0 1 1 1 1 1 2 2 2 2 2]


,test_tuples,pred_tuples,match
0,"[0, 1, 2]","[0, 1, 2]",True
1,"[0, 1, 2]","[0, 1, 2]",True
2,"[0, 1, 2]","[0, 1, 2]",True
3,"[0, 1, 2]","[0, 1, 2]",True
4,"[0, 1, 2]","[0, 1, 2]",True


In [106]:
X_test[y_test!=predicted]

array(['UE performs non-adaptive retransmission of the MAC PDU with redundancy version toggled by one of the last (re)transmission [0,2,3,1 order]',
       'UE inserts a R/R/E/LCID field in the MAC header and a MAC SDU or a MAC control element starts at the next byte',
       'UE responds with a RRCConnectionRequest message in a MAC PDU on UL SCH on CCCH indicating LCID \x9201011\x92'],
      dtype='<U219')

array(['UE in E-UTRA RRC_IDLE state and after transmitting a RRCConnectionRequest message',
       'UE receives a MAC PDU on DL SCH and addressed to its T-CRNTI with value \x9100000\x92B as LCID',
       'UE responds with a RRCConnectionRequest message in a MAC PDU on UL SCH on CCCH indicating LCID \x9201011\x92',
       'UE receives a MAC PDU on DL SCH and addressed to its T-CRNTI but including a reserved value for LCID',
       'UE receives a MAC PDU on DL SCH and addressed to its T-CRNTI with value \x9100000\x92B as LCID',
       'UE receives a Paging message including a matched identity',
       'UE discards the MAC PDU',
       'UE forwards to upper layers the disassembled and demultiplexed SDU on logical channel CCCH',
       'UE responds with a RRCConnectionRequest message in a MAC PDU on UL SCH on CCCH indicating LCID \x9201011\x92'],
      dtype='<U102')

In [87]:
d.head(n=3)

,doc,test_sec,test_name,test_num,with,when,then
0,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,1,UE in E-UTRA RRC_IDLE state and after transmit...,UE receives a MAC PDU on DL SCH and addressed ...,UE discards the MAC PDU
1,36523-1-f00_s07_01,7.1.1.1,CCCH mapped to UL SCH/ DL-SCH / Reserved Logic...,2,UE receives a MAC PDU on DL SCH and addressed ...,UE receives a MAC PDU on DL SCH and addressed ...,UE forwards to upper layers the disassembled a...
2,36523-1-f00_s07_01,7.1.1.1a,CCCH mapped to UL SCH/ DL-SCH / UE Cat 0,1,UE responds with a RRCConnectionRequest messag...,UE receives a Paging message including a match...,UE responds with a RRCConnectionRequest messag...


In [78]:
#%%   COPIED FROM
# https://gitlab.verizon.com/atf/atf-development/atf-data-sciences/-/blob/master/ML_models/atf_xgb_classifier.py

# df2=pd.DataFrame({'sentences':data,'labels':labels})
def compare_classifiers_v2(X_train, y_train, X_test, y_test):
#    VALIDATION_SPLIT = 0.3
    # np.random.seed(seed=seedval)
    # indices = np.arange(d2.shape[0]) # get sequence of row index
    # np.random.shuffle(indices) # shuffle the row indexes
    # data   = d2['sentences'][indices] # shuffle data/product-titles/x-axis
    # labels = d2['labels'][indices]
    # #
    # from sklearn.model_selection import train_test_split
    # X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_fraction, random_state=42)
    
#    #%% COMPARE  TRAIN  AND  TEST  SET DISTRIBUTIONS
#    vdf=pd.concat([y_train.value_counts(normalize=True).round(3) * 100, 
#                   y_test.value_counts(normalize=True).round(3) * 100],
#    sort =False, axis=1,ignore_index=True )
#    vdf.columns = ['train','test']
#    vdf.to_csv(wd+"comp_train_and_test_lables.csv")
    
    #%   ========================================================
    # INITIALIZE OAC SUMMARY TABLE
    # import numpy
    models = ['NB', 'NB+TF', 'NB+TFIDF', 'SVM', 'SVM+TF', 'SVM+TFIDF', 'XGB', 'XGB+TF', 'XGB+TFIDF']
    oacs = np.zeros(len(models))    
    oac_df = pd.DataFrame( {'oac':oacs}, index=models )
    oac_df    
    
    
    #%  DEFINE THREE TRAIN SETS
    from sklearn.feature_extraction.text import CountVectorizer
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_train)   # Plain Train set
    X_test_counts  = count_vect.transform(X_test)
    
    #%
    from sklearn.feature_extraction.text import TfidfTransformer
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)  # TF train set 
    X_test_tf  = tf_transformer.transform(X_test_counts)
    #%
    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)  # TFIDF Train set
    X_test_tfidf  = tfidf_transformer.transform(X_test_counts)
    
    #% XGBoost: XGB 
    import xgboost as xgb
    import warnings
    warnings.filterwarnings(module='xgboost*', action='ignore', category=DeprecationWarning)
    
    #xgbmodel = xgb.XGBClassifier( max_depth=3, n_estimators=300, learning_rate=0.05 ).fit(X_train_counts, y_train)
    xgbmodel = xgb.XGBClassifier( max_depth=1, n_estimators=3000, learning_rate=0.01 ).fit(X_train_counts, y_train)
    predicted = xgbmodel.predict(X_test_counts)
    res=np.mean(predicted == y_test)
    oac_df.loc['XGB']=res
    print('XGB', res)
    
    #% XGBoost: XGB + TF
    #xgbmodel = xgb.XGBClassifier( max_depth=3, n_estimators=300, learning_rate=0.05 ).fit(X_train_tf, y_train)
    xgbmodel = xgb.XGBClassifier( max_depth=1, n_estimators=3000, learning_rate=0.01 ).fit(X_train_tf, y_train)
    predicted = xgbmodel.predict(X_test_tf)
    res=np.mean(predicted == y_test)
    oac_df.loc['XGB+TF']=res
    print('XGB+TF', res)
    
    #% XGB+TFIDF
    #xgbmodel = xgb.XGBClassifier( max_depth=3, n_estimators=300, learning_rate=0.05 ).fit(X_train_tfidf, y_train)
    xgbmodel = xgb.XGBClassifier( max_depth=1, n_estimators=3000, learning_rate=0.01 ).fit(X_train_tfidf, y_train)
    predicted = xgbmodel.predict(X_test_tfidf)
    res=np.mean(predicted == y_test)
    oac_df.loc['XGB+TFIDF']=res
    print('XGB+TFIDF', res)
    
    #% Plain NB Classifier
    from sklearn.naive_bayes import MultinomialNB
    #clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
    from sklearn.pipeline import Pipeline
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        #('tfidf', TfidfTransformer()),
        ('clf', MultinomialNB()),])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['NB']=res
    print('NB', res)
    
    #% NB iwth TF
    # TfidfTransformer(use_idf=False)
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer(use_idf=False)),
        ('clf', MultinomialNB()),])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['NB+TF']=res
    print('NB+TF', res)

    #%
#    lbls=[ x for x in set(labels)]
#    print(lbls)
#    cf = cmdf(y_test, predicted, lbls)
#    cf.to_csv(wd+'xgb_tf_cf.csv')
    
    #% NB with TFIDF
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer(use_idf=True)),
        ('clf', MultinomialNB()),])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['NB+TFIDF']=res
    print('NB+TFIDF', res)


    
    #% Plain SVM
    from sklearn.linear_model import SGDClassifier    
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
    #     ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)), ])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['SVM']=res
    print('SVM', res)

    
    #% SVM with TF
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer(use_idf=False)),
        ('clf',  SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)), ])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['SVM+TF']=res
    print('SVM+TF', res)
    
    #% SVM with TFIDF
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('clf', SGDClassifier(loss='hinge', penalty='l2',
                              alpha=1e-3, random_state=42,
                              max_iter=5, tol=None)), ])
    text_clf.fit(X_train, y_train)
    predicted = text_clf.predict(X_test)
    res=np.mean(predicted == y_test)
    oac_df.loc['SVM+TFIDF']=res
    print('SVM+TFIDF', res)
    
    return oac_df    


In [79]:
oac_df = compare_classifiers_v2(X_train, y_train, X_test, y_test)
oac_df

XGB 0.7777777777777778
XGB+TF 0.7777777777777778
XGB+TFIDF 0.7777777777777778
NB 0.6666666666666666
NB+TF 0.6666666666666666
NB+TFIDF 0.6666666666666666
SVM 0.7777777777777778
SVM+TF 0.7777777777777778
SVM+TFIDF 0.6666666666666666


,oac
NB,0.666667
NB+TF,0.666667
NB+TFIDF,0.666667
SVM,0.777778
SVM+TF,0.777778
SVM+TFIDF,0.666667
XGB,0.777778
XGB+TF,0.777778
XGB+TFIDF,0.777778
